In [ ]:
import random
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import datasets, transforms
from torch import optim

from tqdm import tqdm
import sys

sys.path.append('./codes/')
from EWC_facebook import EWC

%matplotlib inline
import matplotlib.pyplot as plt

plt.style.use("seaborn-white")

In [ ]:
def split_data(data, labels, split, bs, train=True):
    from collections import Counter
    distinct_labels = list(Counter(labels).keys())
    n_labels = len(distinct_labels)
    n_split = int(n_labels/split)
    trans = data.T
    trans.columns = labels
    dataloader = {}
    datasets = {}
    for i in range(n_split):
        cond1 = trans.columns.values >= i*split
        cond2 = trans.columns.values < (i+1)*split
        out = trans.iloc[:,cond1&cond2]
        if train:
            out = out.iloc[:,:200]
        out_label = out.columns.values
        out, out_label = map(torch.tensor, (out.T.to_numpy(), out_label))
        datasets[i] = TensorDataset(out.float(), out_label)
        dataloader[i] = DataLoader(datasets[i], batch_size=bs, shuffle=True)
    
    return (datasets, dataloader)

In [ ]:
def accu(model, dataloader):
    model = model.eval()
    acc = 0
    count = 0
    for input, target in dataloader:
        o = model(input)
        acc += (o.argmax(dim=1).long() == target).float().sum()
        count += len(target)
    return acc/count

In [ ]:
mnist_train = pd.read_csv('./processed_data/mnist_kmnist_train.csv',header=None)
mnist_train_label = pd.read_csv('./processed_data/mnist_kmnist_train_label.csv',header=None)
mnist_train_label = np.array([x[0] for x in np.array(mnist_train_label)])
fmnist_train = pd.read_csv('./processed_data/fmnist_kmnist_train.csv',header=None)
fmnist_train_label = pd.read_csv('./processed_data/fmnist_kmnist_train_label.csv',header=None)
fmnist_train_label = np.array([x[0] for x in np.array(fmnist_train_label)])

In [ ]:
mnist_test = pd.read_csv('./processed_data/mnist_kmnist_test.csv',header=None)
mnist_test_label = pd.read_csv('./processed_data/mnist_kmnist_test_label.csv',header=None)
mnist_test_label = np.array([x[0] for x in np.array(mnist_test_label)])
fmnist_test = pd.read_csv('./processed_data/fmnist_kmnist_test.csv',header=None)
fmnist_test_label = pd.read_csv('./processed_data/fmnist_kmnist_test_label.csv',header=None)
fmnist_test_label = np.array([x[0] for x in np.array(fmnist_test_label)])

In [ ]:
train_data = pd.concat([mnist_train, fmnist_train],axis=0,ignore_index=True)
test_data = pd.concat([mnist_test, fmnist_test],axis=0,ignore_index=True)
train_label = np.concatenate([mnist_train_label, fmnist_train_label+10])
test_label = np.concatenate([mnist_test_label, fmnist_test_label+10])

In [ ]:
train_datasets, trainloader = split_data(train_data, train_label, 2, 64, train=False)
test_datasets, testloader = split_data(test_data, test_label, 2, 64, train=False)

In [ ]:
seeds = [1,2,3,5,8]
n_task = 10

In [ ]:
for seed in seeds:
    args = {
        'n_layers':1,
        'n_hiddens':3200,
        'memory_strength':1000,
        'lr':1e-1,
        'n_memories':1000,
        'cuda':False,
        'seed':seed
    }
    
    gem = EWC(84,20,10,args)
    
    accuracy = np.zeros([n_task, n_task])

    for i in range(n_task):
        for _ in range(1):
            for input, target in tqdm(trainloader[i]):
                gem.observe(input, i, target)

        for j in range(i+1):
            accuracy[i,j] = accu(gem.net,testloader[j]).item()
            
    accuracy = pd.DataFrame(accuracy)
    accuracy.to_csv('./accuracy/EWC_facebook_all_result_lr1e-1_randseed_'+str(seed)+'.csv',index=False,header=False)